Question 1

Let the parameter space $S=[0,1] \times \mathbb{R}^2 \times \mathbb{R}_{++}^2$ and $\lambda = \begin{pmatrix} p & \mu_1 & \mu_2 & \sigma_1 & \sigma_2 \end{pmatrix}^T \in S$. Use:

- $\lambda^0 = \begin{pmatrix} 0.88 & 3.79 & 32.64 & 8.08 & 7.39 \end{pmatrix}^T$
- Fixed step-size $\bar{t} = 0.002$
- Stopping criterion $\epsilon=10^{-5}$
- Maximum number of iterations = 10000

In [69]:
import math
import numpy as np

In [70]:
# This part reads "data.csv"

import pandas as pd
data = pd.read_csv("data.csv", header=None)
df = pd.DataFrame(data).values.tolist()
X = [val for sublist in df for val in sublist]

In [71]:
# A function is defined to calculate the value of a normal pdf
def normalpdf(x,mu,sigma):
    return math.exp(-((x-mu)**2)/(2*sigma**2))/math.sqrt(2*math.pi*sigma**2)

def f(x,p,mu1,mu2,sigma1,sigma2):
    return p*normalpdf(x,mu1,sigma1)+(1-p)*normalpdf(x,mu2,sigma2)

def loglikelihood_f(X,p,mu1,mu2,sigma1,sigma2):
    sum = 0
    for x in X:
        sum += math.log(f(x,p,mu1,mu2,sigma1,sigma2))
    return sum

# Functions are defined for each component of the gradient
def partialp(X,p,mu1,mu2,sigma1,sigma2):
    s = 0
    for x in X:
        s +=  1/f(x,p,mu1,mu2,sigma1,sigma2)*(normalpdf(x,mu1,sigma1)-normalpdf(x,mu2,sigma2))        
    return s    

def partialmu1(X,p,mu1,mu2,sigma1,sigma2):
    s = 0
    for x in X:
        s += 1/f(x,p,mu1,mu2,sigma1,sigma2)*p*normalpdf(x,mu1,sigma1)*(x-mu1)/sigma1**2
    return s

def partialmu2(X,p,mu1,mu2,sigma1,sigma2):
    s = 0
    for x in X:
        s += 1/f(x,p,mu1,mu2,sigma1,sigma2)*(1-p)*normalpdf(x,mu2,sigma2)*(x-mu2)/sigma2**2
    return s

def partialsigma1(X,p,mu1,mu2,sigma1,sigma2):
    s = 0
    for x in X:
        s += 1/f(x,p,mu1,mu2,sigma1,sigma2)*p*normalpdf(x,mu1,sigma1)*((x-mu1)**2-sigma1**2)/abs(sigma1)**3
    return s

def partialsigma2(X,p,mu1,mu2,sigma1,sigma2):
    s = 0
    for x in X:
        s += 1/f(x,p,mu1,mu2,sigma1,sigma2)*(1-p)*normalpdf(x,mu2,sigma2)*((x-mu2)**2-sigma2**2)/abs(sigma2)**3
    return s

# A function is defined to calculate the gradient
def grad(X,p,mu1,mu2,sigma1,sigma2):
    return np.array([partialp(X,p,mu1,mu2,sigma1,sigma2),partialmu1(X,p,mu1,mu2,sigma1,sigma2),partialmu2(X,p,mu1,mu2,sigma1,sigma2),partialsigma1(X,p,mu1,mu2,sigma1,sigma2),partialsigma2(X,p,mu1,mu2,sigma1,sigma2)])

# A function is defined to calculate the projection operator
def proj(p,lb,ub):
    if p > ub:
        p = ub
    elif p < lb:
        p = lb
    return p

In [72]:
# a) Ordinary gradient descent method

# Obtain the initial estimates
p = 0.88
mu1 = 3.79
mu2 = 32.64
sigma1 = 8.08
sigma2 = 7.39

# Set stepsize and stopping criteria as prescribed
tolerance = 10**(-5)
stepsize = 0.002

iteration = 0

while np.linalg.norm(grad(X,p,mu1,mu2,sigma1,sigma2)) > tolerance and iteration <= 10000:
    
    if iteration % 500 == 0 and iteration != 0:
        print('Iteration ',iteration,'; lambda =', np.array([p,mu1,mu2,sigma1,sigma2]))
        
    ascent = np.array([stepsize*partialp(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialmu1(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialmu2(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialsigma1(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialsigma2(X,p,mu1,mu2,sigma1,sigma2)])
    
    if p + ascent[0] < 0 or p + ascent[0] > 1:
        break
    else:
        p += ascent[0]
        mu1 += ascent[1]
        mu2 += ascent[2]
        sigma1 += ascent[3]
        sigma2 += ascent[4]
        iteration += 1     

if iteration <= 10000:
    print('Iteration ',iteration,'; The maximum likelihood estimates of lambda =',np.array([p,mu1,mu2,sigma1,sigma2]))
else:
    print('The algorithm does not converge after 10000 iterations.')

Iteration  4 ; The maximum likelihood estimates of lambda = [ 0.69448322  3.79385472 32.64005472  8.09087593  7.39349867]


Since p is defined to be in interval [0,1], the maximum likelihood estimates of $\lambda$ using the ordinary gradient descent is achieved at the 5th iteration. An unconstrained maximization will cause p to exceed its interval. Therefore, gradient projection method is used below to pull estimate of p back in interval.

In [73]:
# b) Gradient projection method; interval [0,1]; stepsize 0.002

# Obtain the initial estimates
p = 0.88
mu1 = 3.79
mu2 = 32.64
sigma1 = 8.08
sigma2 = 7.39

# Set stepsize and stopping criteria as prescribed
tolerance = 10**(-5)
stepsize = 0.002

iteration = 0
old_lamb = np.zeros(5)
new_lamb = np.array([p,mu1,mu2,sigma1,sigma2])

while np.linalg.norm(old_lamb[0] - new_lamb[0]) > tolerance and iteration <= 10000:
    old_lamb = np.array([p,mu1,mu2,sigma1,sigma2])
    
    if iteration % 500 == 0 and iteration != 0:
        print('Iteration ',iteration,'; lambda =', old_lamb)
        
    ascent = np.array([stepsize*partialp(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialmu1(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialmu2(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialsigma1(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialsigma2(X,p,mu1,mu2,sigma1,sigma2)])
    
    p = proj(p + ascent[0],0,1)
    mu1 += ascent[1]
    mu2 += ascent[2]
    sigma1 += ascent[3]
    sigma2 += ascent[4]
    iteration += 1
    
    new_lamb = np.array([p,mu1,mu2,sigma1,sigma2])

if iteration <= 10000:
    print('Iteration ',iteration,'; The maximum likelihood estimates of lambda =',np.array([p,mu1,mu2,sigma1,sigma2]))
else:
    print('The algorithm does not converge after 10000 iterations.')

Iteration  500 ; lambda = [ 0.86411864  4.77121603 27.14015431  9.52653418 16.72969304]
Iteration  1000 ; lambda = [ 0.8364927   3.94865996 26.97772485  8.37889322 16.21649142]
Iteration  1330 ; The maximum likelihood estimates of lambda = [ 0.83106263  3.70358377 26.8355153   8.21663163 15.84735075]


By pulling the estimate of p back in the interval of [0,1], the maximum likelihood estimates of $\lambda$ converge at iteration 1330.With a better fit of the dataset, the estimates of p and $\sigma_2$ are significantly larger than the unconstrained case, which indicates a larger volume of data fitting better on the left distribution of the normal-normal distribution and a greater spread of data on the right distribution.

In [ ]:
# c) Gradient projection method; interval [0.1,0.9]; stepsize 0.002

# Obtain the initial estimates
p = 0.88
mu1 = 3.79
mu2 = 32.64
sigma1 = 8.08
sigma2 = 7.39

# Set stepsize and stopping criteria as prescribed
tolerance = 10**(-5)
stepsize = 0.002

iteration = 0
old_lamb = np.zeros(5)
new_lamb = np.array([p,mu1,mu2,sigma1,sigma2])

while np.linalg.norm(old_lamb[0] - new_lamb[0]) > tolerance and iteration <= 10000:
    old_lamb = np.array([p,mu1,mu2,sigma1,sigma2])
    
    if iteration % 500 == 0 and iteration != 0:
        print('Iteration ',iteration,'; lambda =', old_lamb)
        
    ascent = np.array([stepsize*partialp(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialmu1(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialmu2(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialsigma1(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialsigma2(X,p,mu1,mu2,sigma1,sigma2)])
    
    p = proj(p + ascent[0],0.1,0.9)
    mu1 += ascent[1]
    mu2 += ascent[2]
    sigma1 += ascent[3]
    sigma2 += ascent[4]
    iteration += 1
    
    new_lamb = np.array([p,mu1,mu2,sigma1,sigma2])

if iteration <= 10000:
    print('Iteration ',iteration,'; The maximum likelihood estimates of lambda =',np.array([p,mu1,mu2,sigma1,sigma2]))
else:
    print('The algorithm does not converge after 10000 iterations.')

Iteration  500 ; lambda = [ 0.84585491  3.93913143 32.56924741  8.32956049  7.77789103]
Iteration  1000 ; lambda = [ 0.84231545  3.94020277 32.48367984  8.32496794  8.00703849]
Iteration  1500 ; lambda = [ 0.83865007  3.92233621 32.37898026  8.31254069  8.16944951]
Iteration  2000 ; lambda = [ 0.83523705  3.90375572 32.26491699  8.30030707  8.29592936]
Iteration  2500 ; lambda = [ 0.83203342  3.88616521 32.14720539  8.28876115  8.401302  ]
Iteration  3000 ; lambda = [ 0.828986    3.86948153 32.02881492  8.27785009  8.49366268]


By constraining the interval of the estimate of p to [0.1,0.9], the maximum likelihood estimates of $\lambda$ do not converge. This is because a stepsize of 0.002 is larger than a stepsize of $\frac{2}{L}$ where L is the Lipschitz constant of $\bigtriangledown f$, causing an oscillating behaviour within the algorithm due to the excessive increase of the constant stepsize gradient method.

In [ ]:
# d) Gradient projection method; interval [0.1,0.9]; stepsize 0.001

# Obtain the initial estimates
p = 0.88
mu1 = 3.79
mu2 = 32.64
sigma1 = 8.08
sigma2 = 7.39

# Set stepsize and stopping criteria as prescribed
tolerance = 10**(-5)
stepsize = 0.001

iteration = 0
old_lamb = np.zeros(5)
new_lamb = np.array([p,mu1,mu2,sigma1,sigma2])

while np.linalg.norm(old_lamb[0] - new_lamb[0]) > tolerance and iteration <= 10000:
    old_lamb = np.array([p,mu1,mu2,sigma1,sigma2])
    
    if iteration % 500 == 0 and iteration != 0:
        print('Iteration ',iteration,'; lambda =', old_lamb)
        
    ascent = np.array([stepsize*partialp(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialmu1(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialmu2(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialsigma1(X,p,mu1,mu2,sigma1,sigma2),
                       stepsize*partialsigma2(X,p,mu1,mu2,sigma1,sigma2)])
    
    p = proj(p + ascent[0],0,1)
    mu1 += ascent[1]
    mu2 += ascent[2]
    sigma1 += ascent[3]
    sigma2 += ascent[4]
    iteration += 1
    
    new_lamb = np.array([p,mu1,mu2,sigma1,sigma2])

if iteration <= 10000:
    print('Iteration ',iteration,'; The maximum likelihood estimates of lambda =',new_lamb)
else:
    print('The algorithm does not converge after 10000 iterations.')
    
print('The log likelihood is',loglikelihood_f(X,p,mu1,mu2,sigma1,sigma2))

By reducing the stepsize to 0.001, the maximum likelihood estimates of $\lambda$  at p $\in$ [0.1, 0.9] converge at iteration 18. Hence, this stepsize contributes to the sufficient decrease of the gradient method.

Question 2

a), b) and c) are available in pdf ("20311298_Assignment 2")

d)

In [ ]:
def proj_affine(x,A,b):
    return x - np.transpose(A)@np.linalg.inv(A@np.transpose(A))@(A@x-b)

In [ ]:
# Gradient projection method

# Obtain the initial estimates
lamb = np.array([[5],[5],[5],[5],[5]])
stepsize = 0.005
grad = np.array([[-120],[-12],[800],[2],[2]])
iteration = 0

A = np.array([[6.5,1,-45,-1,0],[7.8,1,-60,0,-1],[9.2,1,-75,0,0]])
b = np.array([[150],[175],[218]])

while iteration <= 400000:

    if iteration % 20000 == 0 and iteration != 0:
        print('Iteration ',iteration,'; lambda =', lamb)
    
    lamb = proj_affine(lamb + stepsize*grad,A,b)
    for i in range(len(lamb)):
        lamb[i] = max(0,lamb[i])

    iteration += 1
    
profit = (-120*lamb[0] - 12*lamb[1] + 800*lamb[2] + 2*lamb[3] + 2*lamb[4])*1000
print('The dual solution is',profit)
print('The difference between primal and dual is', -2394000 - profit)

$\lambda$ from Primal:

$\lambda = \begin{bmatrix}
0 & 218 & 0 & 68 & 43
\end{bmatrix}^T$


$\lambda$ from Dual at stepsize = 0.005:

$\lambda = \begin{bmatrix}
0 & 215.0688 & 0 & 66.7403 & 42.3746
\end{bmatrix}^T$

$\lambda$ from Dual at stepsize = 0.002:

$\lambda = \begin{bmatrix}
0 & 216.82752 & 0 & 67.49612 & 42.74984
\end{bmatrix}^T$

Similar answer is obtained compared to b), we can improve the precision by applying backtracking instead of constant stepsize. Backtracking ensures iteration of stepsize to obtain suboptimal stepsize until achieving the Sufficient Decrease Property. This guarantees the improvement in the maximization in each iteration with a decreasing stepsize to achieve faster and more accurate convergence. We can also observe that stepsize = 0.002 produces an answer closer to $\lambda$ from Primal.